# Урок 13. Модель BERT и GPT

1. Взять датасет https://huggingface.co/datasets/merionum/ru_paraphraser решить задачу парафраза
2. (дополнительно необязательная задача) на выбор взять https://huggingface.co/datasets/sberquad https://huggingface.co/datasets/blinoff/medical_qa_ru_data натренировать любую модель для вопросно ответной системы как альтернатива можно взять любой NER датасет из https://github.com/natasha/corus#reference и обучить NER

In [1]:
# !pip install transformers sentencepiece datasets

In [2]:
import pandas as pd
import torch
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, BertTokenizer
pd.set_option('display.max_colwidth', None)

corpus = load_dataset('merionum/ru_paraphraser', data_files='test.jsonl')
corpus

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
})

In [3]:
corpus['train'][0,2]

{'id': ['25349', '25382'],
 'id_1': ['34420', '34475'],
 'id_2': ['34421', '34476'],
 'text_1': ['Цены на нефть восстанавливаются',
  'Агент: РФС вновь задерживает зарплату Фабио Капелло'],
 'text_2': ['Парламент Словакии поблагодарил народы бывшего СССР за Победу',
  'СМИ: Агент Фабио Капелло грозится подать в суд на РФС'],
 'class': ['-1', '-1']}

In [4]:
# предобученная модель 'cointegrated/rubert-base-cased-dp-paraphrase-detection'
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)
# if torch.cuda.is_available():
#     model.cuda()

In [5]:
def paraphrase_check(rows, corpus, model, tokenizer):
    res = []
    for row_numb in rows:
        text1 = corpus['train']['text_1'][row_numb]
        text2 = corpus['train']['text_2'][row_numb]
        
        batch = tokenizer(text1, text2, return_tensors='pt') # .to(model.device)
        with torch.inference_mode():
            proba = torch.softmax(model(**batch).logits, -1).numpy()
        
        p = proba[0]
        paraphrase = "парафраз" if p[1] > 0.5 else "другой текст"
        res.append({'Исходный текст': text1, 
                    'Текст сравнения': text2, 
                    # 'Вероятность НЕ парафраза': p[0], 
                    'Вероятность парафраза': p[1], 
                    'Решение': paraphrase})
    return pd.DataFrame(res)

In [6]:
paraphrase_check(rows=[0, 11, 2, 3, 4, 5], 
                 corpus=corpus, 
                 model=model,
                 tokenizer=tokenizer)

,Исходный текст,Текст сравнения,Вероятность парафраза,Решение
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего СССР за Победу,0.054301,другой текст
1,МЧС РФ: тела погибших российских дипломатов доставят из Непала 11 мая,Тела погибших в Непале российских дипломатов доставят на родину 11 мая,0.984235,парафраз
2,Агент: РФС вновь задерживает зарплату Фабио Капелло,СМИ: Агент Фабио Капелло грозится подать в суд на РФС,0.120681,другой текст
3,День Победы в Москве обещает выдаться облачным,Любляна отпразднует День Победы вместе с Москвой,0.053447,другой текст
4,Посол РФ в США: Россия будет бороться с попытками переписать историю,Правительство запланировало заработать на лотереях 400 млрд руб.,0.053032,другой текст
5,Вертолет с 11 иностранцами на борту упал в Пакистане,В Пакистане упал вертолет с 11 иностранцами,0.982860,парафраз
